In [ ]:
import os

In [2]:
print(os.getcwd())

e:\STUDY\TENSORFLOW\Projects\1_CNN_Project\research


In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'e:\\STUDY\\TENSORFLOW\\Projects\\1_CNN_Project'

In [5]:
# Update the src/cnn_classifier/entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBasseModelConfig:

    root_dir: Path
    base_model_path : Path
    custom_base_model_path: Path
    params_include_top : bool
    params_weights : str
    params_image_size : list
    params_learning_rate : float
    params_classes : int
    

In [6]:
# Update the src/cnn_classifier/config/configuration.

from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:

    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        # Creating directory
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBasseModelConfig:

        config=self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config= PrepareBasseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            custom_base_model_path=Path(config.custom_base_model_path),
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_image_size=self.params.INPUT_SHAPE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_classes=self.params.CLASSES
                )
        
        return prepare_base_model_config

In [8]:
# Update the src/cnn_classifier/components.

import os
from urllib import request
from zipfile import ZipFile
import tensorflow as tf


In [9]:
class PrepareBaseModel:

    def __init__(self, config:PrepareBasseModelConfig):

        self.config=config

    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):

        model.save(path)

    
    def get_base_model(self):

        self.model=tf.keras.applications.vgg16.VGG16(
            include_top=self.config.params_include_top,
            weights=self.config.params_weights,
            input_shape=self.config.params_image_size
        )

        # calling static method(save_model) to save the base model
        self.save_model(path=self.config.base_model_path,
                        model=self.model)
        
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all:bool, freeze_till:int, learning_rate, input_shape):

        # Freeze all layers if freeze_all is True
        if freeze_all:
            model.trainable = False  # Correctly set layer.trainable
        
        # Freeze layers up to a specific number if freeze_till is provided
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False  # Correctly set layer.trainable

        # Creating new model on top
        inputs=tf.keras.Input(shape=input_shape, name="input_layer")

        # Apply Rescaling
        rescale_layer=tf.keras.layers.Rescaling(scale=1/255., name="rescaling_layer")
        x = rescale_layer(inputs)

        # VGG16 model
        x = model(x, training=False)

        # Hidden layers
        flatten_layer = tf.keras.layers.Flatten(name="flatten_layer")(x)

        # hidden_layer1 = tf.keras.layers.Dense(
        #                                     units=10,
        #                                     activation="sigmoid",
        #                                     name="hidden_layer1"
        #                                     )(flatten_layer)

        # hidden_layer2=tf.keras.layers.Dense(units=10, activation="relu", name="hidden_layer2")(hidden_layer1)

        #output_layer=tf.keras.layers.Dense(units=1, activation="sigmoid", name="output_layer")(hidden_layer2)

        hidden_layer1=tf.keras.layers.Dense(units=5, activation="relu", name="hidden_layer1")(flatten_layer)

        output_layer=tf.keras.layers.Dense(units=1, activation="sigmoid", name="output_layer")(hidden_layer1)

        full_model = tf.keras.models.Model(
                                  inputs=inputs,
                                  outputs=output_layer
                                  )
        
        full_model.summary()
        
        return full_model

        
    def custom_base_model(self):

        self.full_model=self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate,
            input_shape=self.config.params_image_size

            )
        
        self.save_model(path=self.config.custom_base_model_path,
                        model=self.full_model)


In [10]:
try:
    config=ConfigurationManager()
    prepare_base_model_config=config.get_prepare_base_model_config()
    prepare_base_model=PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.custom_base_model()

except Exception as e:
    raise e

[2024-12-25 18:41:44,323 : INFO : common : yaml file : config\config.yaml loaded successfully]
[2024-12-25 18:41:44,383 : INFO : common : yaml file : params.yaml loaded successfully]
[2024-12-25 18:41:44,388 : INFO : common : Created directory at : artifacts]
[2024-12-25 18:41:44,391 : INFO : common : Created directory at : artifacts/prepare_base_model]
[2024-12-25 18:41:48,703 : WARNING : saving_utils : Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "[<keras.layers.reshaping.flatten.Flatten object at 0x000002C2A9287FA0>, <keras.layers.core.dense.Dense object at 0x000002C2A925A800>, <keras.layers.core.dense.Dense object at 0x000002C2A925A770>, <keras.layers.core.dense.Dense object at 0x000002C2A925A290>]"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 